In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3]

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = 0.0032
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_1688\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 132.88 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX1
            dataY = dataY1
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
        else:
            #5s
            dataX = dataX5
            dataY = dataY5
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.5385684967041016 Train_acc: 4.150000%
Train Epoch: 2/60 Train_Loss: 3.2309417724609375 Train_acc: 8.587501%
Train Epoch: 3/60 Train_Loss: 3.713115930557251 Train_acc: 12.412500%
Train Epoch: 4/60 Train_Loss: 2.9869179725646973 Train_acc: 13.587501%
Train Epoch: 5/60 Train_Loss: 2.5442092418670654 Train_acc: 18.275002%
Train Epoch: 6/60 Train_Loss: 2.3684818744659424 Train_acc: 21.450001%
Train Epoch: 7/60 Train_Loss: 2.411705255508423 Train_acc: 24.550001%
Train Epoch: 8/60 Train_Loss: 2.842207193374634 Train_acc: 26.200001%
Train Epoch: 9/60 Train_Loss: 2.507023334503174 Train_acc: 27.975000%
Train Epoch: 10/60 Train_Loss: 2.5918681621551514 Train_acc: 30.162502%
Train Epoch: 11/60 Train_Loss: 2.072483

Train Epoch: 47/60 Train_Loss: 0.47527995705604553 Train_acc: 87.604172%
Train Epoch: 48/60 Train_Loss: 0.6722420454025269 Train_acc: 87.458336%
Train Epoch: 49/60 Train_Loss: 0.3086916208267212 Train_acc: 87.958336%
Train Epoch: 50/60 Train_Loss: 0.5573756098747253 Train_acc: 87.791672%
Train Epoch: 51/60 Train_Loss: 0.42347416281700134 Train_acc: 88.208336%
Train Epoch: 52/60 Train_Loss: 0.38437771797180176 Train_acc: 87.666672%
Train Epoch: 53/60 Train_Loss: 0.3825590908527374 Train_acc: 88.500000%
Train Epoch: 54/60 Train_Loss: 0.5741106867790222 Train_acc: 88.083336%
Train Epoch: 55/60 Train_Loss: 0.4176889657974243 Train_acc: 88.583336%
Train Epoch: 56/60 Train_Loss: 0.17738430202007294 Train_acc: 88.562500%
Train Epoch: 57/60 Train_Loss: 0.35546526312828064 Train_acc: 88.604172%
Train Epoch: 58/60 Train_Loss: 0.5721688866615295 Train_acc: 88.770836%
Train Epoch: 59/60 Train_Loss: 0.5154345035552979 Train_acc: 87.812500%
Train Epoch: 60/60 Train_Loss: 0.35796409845352173 Train_ac

Train Epoch: 30/60 Train_Loss: 1.7150789499282837 Train_acc: 54.575001%
Train Epoch: 31/60 Train_Loss: 0.9832909107208252 Train_acc: 57.900002%
Train Epoch: 32/60 Train_Loss: 1.5612208843231201 Train_acc: 57.737503%
Train Epoch: 33/60 Train_Loss: 1.4607617855072021 Train_acc: 58.737503%
Train Epoch: 34/60 Train_Loss: 1.381496548652649 Train_acc: 58.200005%
Train Epoch: 35/60 Train_Loss: 1.2214303016662598 Train_acc: 59.125004%
Train Epoch: 36/60 Train_Loss: 1.1351940631866455 Train_acc: 59.900002%
Train Epoch: 37/60 Train_Loss: 1.3290112018585205 Train_acc: 59.412502%
Train Epoch: 38/60 Train_Loss: 1.2218984365463257 Train_acc: 59.912502%
Train Epoch: 39/60 Train_Loss: 1.2212989330291748 Train_acc: 60.050003%
Train Epoch: 40/60 Train_Loss: 1.5124118328094482 Train_acc: 60.775002%
Train Epoch: 41/60 Train_Loss: 1.3877217769622803 Train_acc: 60.512505%
Train Epoch: 42/60 Train_Loss: 0.9383658170700073 Train_acc: 61.225002%
Train Epoch: 43/60 Train_Loss: 1.2185368537902832 Train_acc: 61.6

Train Epoch: 14/60 Train_Loss: 0.31958743929862976 Train_acc: 86.062500%
Train Epoch: 15/60 Train_Loss: 0.5757293701171875 Train_acc: 86.312500%
Train Epoch: 16/60 Train_Loss: 0.14687421917915344 Train_acc: 90.000000%
Train Epoch: 17/60 Train_Loss: 0.1803346872329712 Train_acc: 93.437500%
Train Epoch: 18/60 Train_Loss: 0.12584568560123444 Train_acc: 93.687500%
Train Epoch: 19/60 Train_Loss: 0.1986227184534073 Train_acc: 94.250000%
Train Epoch: 20/60 Train_Loss: 0.13242174685001373 Train_acc: 93.687500%
Train Epoch: 21/60 Train_Loss: 0.29297804832458496 Train_acc: 94.187500%
Train Epoch: 22/60 Train_Loss: 0.11759292334318161 Train_acc: 93.500000%
Train Epoch: 23/60 Train_Loss: 0.17934486269950867 Train_acc: 94.187500%
Train Epoch: 24/60 Train_Loss: 0.2616865932941437 Train_acc: 92.812500%
Train Epoch: 25/60 Train_Loss: 0.14137893915176392 Train_acc: 95.187500%
Train Epoch: 26/60 Train_Loss: 0.21840900182724 Train_acc: 93.750000%
Train Epoch: 27/60 Train_Loss: 0.13073448836803436 Train_a

Train Epoch: 1/60 Train_Loss: 1.4956579208374023 Train_acc: 56.291668%
Train Epoch: 2/60 Train_Loss: 1.5080047845840454 Train_acc: 64.750000%
Train Epoch: 3/60 Train_Loss: 1.1781364679336548 Train_acc: 63.729168%
Train Epoch: 4/60 Train_Loss: 1.1600230932235718 Train_acc: 63.937500%
Train Epoch: 5/60 Train_Loss: 1.179890513420105 Train_acc: 66.395836%
Train Epoch: 6/60 Train_Loss: 1.072716236114502 Train_acc: 67.125000%
Train Epoch: 7/60 Train_Loss: 0.6494367718696594 Train_acc: 67.479172%
Train Epoch: 8/60 Train_Loss: 1.0191072225570679 Train_acc: 67.395836%
Train Epoch: 9/60 Train_Loss: 0.7306202054023743 Train_acc: 68.916672%
Train Epoch: 10/60 Train_Loss: 1.1982966661453247 Train_acc: 68.250000%
Train Epoch: 11/60 Train_Loss: 1.0154632329940796 Train_acc: 68.687500%
Train Epoch: 12/60 Train_Loss: 0.8975143432617188 Train_acc: 70.145836%
Train Epoch: 13/60 Train_Loss: 0.9079357981681824 Train_acc: 67.354172%
Train Epoch: 14/60 Train_Loss: 1.0804598331451416 Train_acc: 66.437500%
Tra

Train Epoch: 50/60 Train_Loss: 0.027330072596669197 Train_acc: 98.875000%
Train Epoch: 51/60 Train_Loss: 0.03723973035812378 Train_acc: 98.875000%
Train Epoch: 52/60 Train_Loss: 0.09938989579677582 Train_acc: 99.125000%
Train Epoch: 53/60 Train_Loss: 0.07775166630744934 Train_acc: 98.875000%
Train Epoch: 54/60 Train_Loss: 0.08764322847127914 Train_acc: 99.000000%
Train Epoch: 55/60 Train_Loss: 0.06690321862697601 Train_acc: 99.187500%
Train Epoch: 56/60 Train_Loss: 0.05140797421336174 Train_acc: 99.062500%
Train Epoch: 57/60 Train_Loss: 0.030906716361641884 Train_acc: 98.562500%
Train Epoch: 58/60 Train_Loss: 0.06848384439945221 Train_acc: 99.000000%
Train Epoch: 59/60 Train_Loss: 0.058270275592803955 Train_acc: 99.187500%
Train Epoch: 60/60 Train_Loss: 0.04426871985197067 Train_acc: 99.125000%
Test accuracy of the model: 32.100000%
Test accuracy of the model: 44.833333%
Test accuracy of the model: 50.500000%
Training Time: 33.24 seconds
可用的 GPU 数量: 1
----------------------------------

Train Epoch: 34/60 Train_Loss: 0.4578688144683838 Train_acc: 85.041672%
Train Epoch: 35/60 Train_Loss: 0.5479956269264221 Train_acc: 85.083336%
Train Epoch: 36/60 Train_Loss: 0.6220570206642151 Train_acc: 84.958336%
Train Epoch: 37/60 Train_Loss: 0.513839840888977 Train_acc: 85.437500%
Train Epoch: 38/60 Train_Loss: 0.3895396292209625 Train_acc: 85.604172%
Train Epoch: 39/60 Train_Loss: 0.42213189601898193 Train_acc: 84.437500%
Train Epoch: 40/60 Train_Loss: 0.5817120671272278 Train_acc: 86.520836%
Train Epoch: 41/60 Train_Loss: 0.46999651193618774 Train_acc: 84.687500%
Train Epoch: 42/60 Train_Loss: 0.4217873513698578 Train_acc: 85.625000%
Train Epoch: 43/60 Train_Loss: 0.6107743382453918 Train_acc: 84.875000%
Train Epoch: 44/60 Train_Loss: 0.5639724135398865 Train_acc: 86.229172%
Train Epoch: 45/60 Train_Loss: 0.6585450768470764 Train_acc: 86.541672%
Train Epoch: 46/60 Train_Loss: 0.3766450583934784 Train_acc: 87.895836%
Train Epoch: 47/60 Train_Loss: 0.3741718828678131 Train_acc: 88

Train Epoch: 17/60 Train_Loss: 1.4245164394378662 Train_acc: 47.900002%
Train Epoch: 18/60 Train_Loss: 1.921303153038025 Train_acc: 49.162502%
Train Epoch: 19/60 Train_Loss: 1.6131322383880615 Train_acc: 49.837502%
Train Epoch: 20/60 Train_Loss: 1.3662997484207153 Train_acc: 49.412502%
Train Epoch: 21/60 Train_Loss: 1.7521735429763794 Train_acc: 50.362503%
Train Epoch: 22/60 Train_Loss: 1.6875745058059692 Train_acc: 51.987503%
Train Epoch: 23/60 Train_Loss: 1.5067636966705322 Train_acc: 52.212502%
Train Epoch: 24/60 Train_Loss: 1.517403483390808 Train_acc: 53.475002%
Train Epoch: 25/60 Train_Loss: 1.5135278701782227 Train_acc: 54.000004%
Train Epoch: 26/60 Train_Loss: 1.3003904819488525 Train_acc: 54.212502%
Train Epoch: 27/60 Train_Loss: 1.445314645767212 Train_acc: 54.700001%
Train Epoch: 28/60 Train_Loss: 1.509824275970459 Train_acc: 55.225002%
Train Epoch: 29/60 Train_Loss: 1.506078839302063 Train_acc: 55.987503%
Train Epoch: 30/60 Train_Loss: 1.4739817380905151 Train_acc: 55.78750

Train Epoch: 1/60 Train_Loss: 0.9737563133239746 Train_acc: 61.250000%
Train Epoch: 2/60 Train_Loss: 0.839393138885498 Train_acc: 76.812500%
Train Epoch: 3/60 Train_Loss: 0.7754669189453125 Train_acc: 82.687500%
Train Epoch: 4/60 Train_Loss: 0.537550151348114 Train_acc: 81.375000%
Train Epoch: 5/60 Train_Loss: 0.4956716299057007 Train_acc: 83.937500%
Train Epoch: 6/60 Train_Loss: 0.5812451243400574 Train_acc: 84.187500%
Train Epoch: 7/60 Train_Loss: 0.3393666446208954 Train_acc: 84.812500%
Train Epoch: 8/60 Train_Loss: 0.5713579058647156 Train_acc: 87.125000%
Train Epoch: 9/60 Train_Loss: 0.6404893398284912 Train_acc: 85.875000%
Train Epoch: 10/60 Train_Loss: 0.6712005138397217 Train_acc: 84.562500%
Train Epoch: 11/60 Train_Loss: 0.8226726055145264 Train_acc: 85.437500%
Train Epoch: 12/60 Train_Loss: 0.692047655582428 Train_acc: 82.062500%
Train Epoch: 13/60 Train_Loss: 0.9046320915222168 Train_acc: 81.812500%
Train Epoch: 14/60 Train_Loss: 0.7799810171127319 Train_acc: 78.687500%
Trai

In [9]:
def getMax(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][len(data[fold])-1])
        
    return max(TL2)

def getMin(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][len(data[fold])-1])
        
    return min(TL2)

def getMean(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][len(data[fold])-1])
        
    return np.mean(TL2)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMax(kFoldTrainAccuracy), getMax(kFoldTestAccuracy1), getMax(kFoldTestAccuracy3), getMax(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMin(kFoldTrainAccuracy), getMin(kFoldTestAccuracy1), getMin(kFoldTestAccuracy3), getMin(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMean(kFoldTrainAccuracy), getMean(kFoldTestAccuracy1), getMean(kFoldTestAccuracy3), getMean(kFoldTestAccuracy5)))

f.close()